In [1]:
desktop

In [3]:
addpath ~
addpath ~/npl
startupJupyter
pierreSettingsEric

# Construct the LFADS Run manager objects

In [36]:
% set these to the appropriate paths
lfadsRoot = getenv('LFADS_DATA_ROOT');
lfadsExportRoot = fullfile(lfadsRoot, 'export_v05_broadbandRethreshNonSorted_filtered');
lfadsRunRoot = fullfile(lfadsRoot, 'runs');

assert(exist(lfadsExportRoot, 'dir') > 0)
assert(exist(lfadsRunRoot, 'dir') > 0)

In [37]:
dc = PierreEricLFADS.DatasetCollection(lfadsExportRoot);
dc.context = 'Eric_v5';
dc.autoDetectDatasets();
dc.loadInfo();

Loading info :      0% [               ]    2.3% [               ]    4.5% [               ]    6.8% [_              ]    9.1% [_              ]   11.4% [_              ]   13.6% [__             ]   15.9% [__             ]   18.2% [__             ]   20.5% [___            ]   22.7% [___            ]     25% [___            ]   27.3% [____           ]   29.5% [____           ]   31.8% [____           ]   34.1% [_____          ]   36.4% [_____          ]   38.6% [_____          ]   40.9% [______         ]   43.2% [______         ]   45.5% [______         ]   47.7% [_______        ]     50% [_______        ]   52.3% [_______        ]   54.5% [________       ]   56.8% [________       ]   59.1% [________       ]   61.4% [_________      ]   63.6% [_________      ]   65.9% [_________      ]   68.2% [__________     ]   70.5% [__________     ]   72.7% [__________     ]     75% [___________    ]   77.3% [___________    ]   79.5% [___________    ]   81.8% [____________   ]   84.1% [____________  

In [38]:
dc


dc = 

PierreEricLFADS.DatasetCollection "export_v05_broadbandRethreshNonSorted_filtered"
  44 datasets in /data6/Pierre42/lfads/export_v05_broadbandRethreshNonSorted_filtered
  [ 1] PierreEricLFADS.Dataset "subject_Pierre.date_2016-08-19.saveTagGroup_1.saveTag_2_3_5_export"
  [ 2] PierreEricLFADS.Dataset "subject_Pierre.date_2016-08-23.saveTagGroup_1.saveTag_1_2_3_4_5_7_export"
  [ 3] PierreEricLFADS.Dataset "subject_Pierre.date_2016-08-24.saveTagGroup_1.saveTag_1_2_3_4_5_6_7_8_9_10_11_12_export"
  [ 4] PierreEricLFADS.Dataset "subject_Pierre.date_2016-08-25.saveTagGroup_1.saveTag_1_2_3_4_5_6_7_8_9_10_11_export"
  [ 5] PierreEricLFADS.Dataset "subject_Pierre.date_2016-08-26.saveTagGroup_1.saveTag_1_2_3_4_5_6_7_8_9_10_11_12_14_export"
  [ 6] PierreEricLFADS.Dataset "subject_Pierre.date_2016-08-31.saveTagGroup_1.saveTag_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18_export"
  [ 7] PierreEricLFADS.Dataset "subject_Pierre.date_2016-09-01.saveTagGroup_1.saveTag_3_4_5_6_7_8_9_10_11_12_13_14_ex

In [40]:
% Configure hyperparameters
par = PierreEricLFADS.RunParams;
par.version = 20171107;
par.c_batch_size = 40;

par.nTrialsKeep = Inf;
par.spikeBinMs = 10;
par.preKeep = 500;
par.postKeep = 700;
par.align = 'GoCue';

par.useAlignmentMatrix = true;
par.useSingleDatasetAlignmentMatrix = false;

par.c_factors_dim = 8;
par.c_ic_enc_dim = 100;
par.c_ci_enc_dim = 100;
par.c_gen_dim = 100;

par.c_learning_rate_stop = 1e-3;
par.c_l2_increase_steps = 900;
par.c_kl_increase_steps = 900;
par.c_learning_rate_decay_factor = 0.98;
par.c_co_dim = 0; % no controller
par.c_factors_dim = 16;
par.posterior_mean_kind = 'posterior_sample_and_average';
par.num_samples_posterior = 512;

par.c_do_train_readin = false; % we used a fixed readin matrix for the paper, setting this to true also works

In [41]:
% Datasets have been already filtered for you, but this code was run:
% dc.filterHavingMinimumTrials(ceil(par.c_batch_size * (par.trainToTestRatio+1)));
% dc.filterHasHighSNRChannels();

In [42]:
%% Build run Collection
rc = PierreEricLFADS.RunCollection(lfadsRunRoot, 'singleVsStitched_v11', dc);
rc.addParams(par);

cls = 'PierreEricLFADS.Run';
for iR = 1:dc.nDatasets
    rc.addRunSpec(LFADS.RunSpec(dc.datasets(iR).getSingleRunName(), cls, dc, iR));
end

% stitched with all datasets
rc.addRunSpec(LFADS.RunSpec('all', cls, dc, 1:dc.nDatasets));

In [42]:
rc


rc = 

PierreEricLFADS.RunCollection "singleVsStitched_v11" (45 runs total)
  Dataset Collection "export_v05_broadbandRethreshNonSorted_filtered" (44 datasets) in /data6/Pierre42/lfads/export_v05_broadbandRethreshNonSorted_filtered
  Path: /data6/Pierre42/lfads/runs/singleVsStitched_v11

  1 parameter settings
  [1 param_39KY_p data_bPkSOi] PierreEricLFADS.RunParams "" align=GoCue preKeep=500 postKeep=700 nTrialsKeep=Inf spikeBinMs=10 c_learning_rate_stop=0.001 c_batch_size=40 c_co_dim=0 c_ci_enc_dim=100 c_ic_enc_dim=100 c_factors_dim=16 c_do_train_readin=false useAlignmentMatrix=true

  45 run specifications
  [ 1] LFADS.RunSpec "single_subject_Pierre.date_2016-08-19.saveTagGroup_1.saveTag_2_3_5_export" (1 datasets)
  [ 2] LFADS.RunSpec "single_subject_Pierre.date_2016-08-23.saveTagGroup_1.saveTag_1_2_3_4_5_7_export" (1 datasets)
  [ 3] LFADS.RunSpec "single_subject_Pierre.date_2016-08-24.saveTagGroup_1.saveTag_1_2_3_4_5_6_7_8_9_10_11_12_export" (1 datasets)
  [ 4] LFADS.RunSpec "sin

# Export for LFADS 

In [44]:
rc.prepareForLFADS();
rc.writeSummaryText();
rc.writeTensorboardShellScript();
rc.writeShellScriptRunQueue('display', 0);

Generating input data for each run :      0% [               ]    2.2% [               ]    4.4% [               ]    6.7% [_              ]    8.9% [_              ]   11.1% [_              ]   13.3% [_              ]   15.6% [__             ]   17.8% [__             ]     20% [___            ]   22.2% [___            ]   24.4% [___            ]   26.7% [____           ]   28.9% [____           ]   31.1% [____           ]   33.3% [____           ]   35.6% [_____          ]   37.8% [_____          ]     40% [______         ]   42.2% [______         ]   44.4% [______         ]   46.7% [_______        ]   48.9% [_______        ]   51.1% [_______        ]   53.3% [_______        ]   55.6% [________       ]   57.8% [________       ]     60% [_________      ]   62.2% [_________      ]   64.4% [_________      ]   66.7% [__________     ]   68.9% [__________     ]   71.1% [__________     ]   73.3% [__________     ]   75.6% [___________    ]   77.8% [___________    ]     80% [____________   ]  

# Run LFADS

Here you would run the LFADS run queue bash script located in

```
/data6/Pierre42/lfads/runs/singleVsStitched_v11/run_lfadsqueue.py
```

For reference, see https://lfads.github.io/lfads-run-manager/running/

# Load in the results of LFADS

In [45]:
rc.loadPosteriorMeans()

Loading posterior mean samples :      0% [               ]   Loading posterior means for each dataset :      0% [               ]    100% [_______________]   
   2.2% [               ]   Loading posterior means for each dataset :      0% [               ]    100% [_______________]   
   4.4% [               ]   Loading posterior means for each dataset :      0% [               ]    100% [_______________]   
   6.7% [_              ]   Loading posterior means for each dataset :      0% [               ]    100% [_______________]   
   8.9% [_              ]   Loading posterior means for each dataset :      0% [               ]    100% [_______________]   
  11.1% [_              ]   Loading posterior means for each dataset :      0% [               ]    100% [_______________]   
  13.3% [_              ]   Loading posterior means for each dataset :      0% [               ]    100% [_______________]   
  15.6% [__             ]   Loading posterior means for each dataset :      0% [     

In [46]:
rc.runs(1).posteriorMeans


ans = 

  PosteriorMeans with properties:

                        kind: 'posterior_sample_and_average'
                        time: [1×120 double]
          controller_outputs: []
                     factors: [16×120×212 double]
                post_g0_mean: [64×212 double]
              post_g0_logvar: [64×212 double]
               generator_ics: [100×212 double]
            generator_states: [100×120×212 double]
                       rates: [24×120×212 double]
                       costs: [212×1 double]
              nll_bound_vaes: [212×1 double]
             nll_bound_iwaes: [212×1 double]
                   validInds: [43×1 uint16]
                   trainInds: [169×1 double]
                      params: [1×1 PierreEricLFADS.RunParams]
                conditionIds: {212×1 cell}
                   rawCounts: [24×1200×212 double]
              externalInputs: []
         datasetIndexByTrial: [212×1 double]
                  binWidthMs: 10
    scaleFactorCountsToRates: 100
  

In [47]:
rc.exportResultsToH5()

Exporting posterior means as HDF5 :      0% [               ]   Exporting to /data6/Pierre42/lfads/runs/singleVsStitched_v11/posterior_means_export/single_subject_Pierre.date_2016-08-19.saveTagGroup_1.saveTag_2_3_5_export
Exporting Posterior Means :      0% [               ]    100% [_______________]   
   2.2% [               ]   Exporting to /data6/Pierre42/lfads/runs/singleVsStitched_v11/posterior_means_export/single_subject_Pierre.date_2016-08-23.saveTagGroup_1.saveTag_1_2_3_4_5_7_export
Exporting Posterior Means :      0% [               ]    100% [_______________]   
   4.4% [               ]   Exporting to /data6/Pierre42/lfads/runs/singleVsStitched_v11/posterior_means_export/single_subject_Pierre.date_2016-08-24.saveTagGroup_1.saveTag_1_2_3_4_5_6_7_8_9_10_11_12_export
Exporting Posterior Means :      0% [               ]    100% [_______________]   
   6.7% [_              ]   Exporting to /data6/Pierre42/lfads/runs/singleVsStitched_v11/posterior_means_export/single_subject_Pie